In [ ]:

import dpkt
import struct
import typing
import argparse
import math
import matplotlib.pyplot as plt
import numpy as np
import sys
import time


def get_udp_payload_bytes(pcap_filename) -> typing.List[bytes]:
    """
    Read UDP payload bytes from a .pcap(ng) file
    return list with one entry per packet, entry containing udp payload
    """
    timestamps = []
    try:
        with open(pcap_filename, "rb") as f:
            pcap_read = dpkt.pcap.UniversalReader(f)
            udp_payloads = list()
            for ts, buf in pcap_read:
                # skip packet if not long enough to contain IP+UDP+CODIF hdrs
                if len(buf) < (34 + 8 + 64):
                    print(f"WARNING: Found packet that is too small {len(buf)}Bytes")
                    continue
                eth = dpkt.ethernet.Ethernet(buf)
                ip = eth.data
                # skip non-UDP packets
                if ip.p != dpkt.ip.IP_PROTO_UDP:
                    print(f"WARNING: Found packet that is not UDP {ip.p} type")
                    continue
                # add the UDP payload data into the list of payloads
                udp = ip.data
                udp_payloads.append(udp.data)
                timestamps.append(ts)
    except FileNotFoundError as fnf_err:
        print(fnf_err)
        sys.exit(1)

    return timestamps, udp_payloads


N_POL = 2
N_VALS_PER_CPLX = 2
N_BYES_PER_VAL = 1
N_BYTES_PER_SAMPLE = N_POL * N_VALS_PER_CPLX * N_BYES_PER_VAL


def get_channel_power(beam, payload):
    seq_no = struct.unpack("Q", payload[0:8])[0]
    scale1 = struct.unpack("f", payload[32:36])[0]
    first_chan = struct.unpack("I", payload[48:52])[0]
    num_chan = struct.unpack("H", payload[52:54])[0]
    valid_chan = struct.unpack("H", payload[54:56])[0]
    num_sample = struct.unpack("H", payload[56:58])[0]
    beam_id = struct.unpack("H", payload[58:60])[0]
    sample_per_weight = struct.unpack("B", payload[67:68])[0]
    if beam_id != beam:
        return None
    weights_offset = 96  # start of weights (multiple of 16bytes=128 bits)
    n_weight_bytes = num_sample / sample_per_weight * 2 * num_chan
    # weights padded to multiple of 128 bits = 16 bytes
    data_offset = weights_offset + math.ceil(n_weight_bytes / 16) * 16

    np_chanl_pwr = np.zeros((2, num_chan))
    for ch in range(0, valid_chan):
        ch_offset = data_offset + (
            ch * num_sample * N_BYES_PER_VAL * N_VALS_PER_CPLX * N_POL
        )
        # sum sample power for both polarisations
        for pol in range(0, 2):
            pol_base = ch_offset + pol * num_sample * N_BYES_PER_VAL * N_VALS_PER_CPLX
            for sample_idx in range(0, num_sample):
                loc_x = pol_base + sample_idx * 2
                x_i, x_q = struct.unpack("bb", payload[loc_x : loc_x + 2])
                sample_pwr = x_i * x_i + x_q * x_q
                np_chanl_pwr[pol][ch] += sample_pwr
    # average power per-complex-sample per channel
    np_chanl_pwr = np_chanl_pwr / (scale1 * scale1) / valid_chan / num_sample
    # print(f"scale: {scale1}")
    # print(f"weights_bytes: {n_weight_bytes} data_offset: {data_offset} + {valid_chan*pol*num_sample}samples")
    # print(f"pol_base {pol_base}+{num_sample*2}")

    return (seq_no, first_chan, np_chanl_pwr)


def get_packet_data(payload):
    seq_no = struct.unpack("Q", payload[0:8])[0]
    sample_no = struct.unpack("Q", payload[8:16])[0]
    scale1 = struct.unpack("f", payload[32:36])[0]
    first_chan = struct.unpack("I", payload[48:52])[0]
    num_chan = struct.unpack("H", payload[52:54])[0]
    valid_chan = struct.unpack("H", payload[54:56])[0]
    num_sample = struct.unpack("H", payload[56:58])[0]
    beam_id = struct.unpack("H", payload[58:60])[0]
    sample_per_weight = struct.unpack("B", payload[67:68])[0]

    weights_offset = 96  # start of weights (multiple of 16bytes=128 bits)
    n_weight_bytes = num_sample / sample_per_weight * 2 * num_chan
    # weights padded to multiple of 128 bits = 16 bytes
    data_offset = weights_offset + math.ceil(n_weight_bytes / 16) * 16
    # samples for one packet = (channels) x (2 pol) x (time samples)
    samples = np.zeros((valid_chan, 2, num_sample), dtype=np.complex64)

    for ch in range(0, valid_chan):
        ch_offset = data_offset + (
            ch * num_sample * N_BYES_PER_VAL * N_VALS_PER_CPLX * N_POL
        )
        for pol in range(0, 2):
            pol_base = ch_offset + pol * num_sample * N_BYES_PER_VAL * N_VALS_PER_CPLX
            for sample_idx in range(0, num_sample):
                loc_x = pol_base + sample_idx * 2
                x_i, x_q = struct.unpack("bb", payload[loc_x : loc_x + 2])
                samples[ch, pol, sample_idx] = 1j * np.float32(x_q) + np.float32(x_i)
    return (seq_no, sample_no, beam_id, first_chan, scale1, samples)




lambda_file = "cap_13Dec2024_0.pcapng"
total_ADCs = 20
pcount_max = 900

# read pcap file
tstamps, payloads = get_udp_payload_bytes(lambda_file)

# run through the data to find the number of beams and channels
first_pkt = True
start_seq_no = 0
start_chan = 0
end_seq_no = 0
end_chan = 0
total_packets = 0

for ts, pkt_payload in zip(tstamps, payloads):
    seq_no = struct.unpack("<Q", pkt_payload[0:8])[0]
    FPGA_id = struct.unpack("<I", pkt_payload[8:12])[0]
    freq_chan = struct.unpack("<H", pkt_payload[12:14])[0]
    total_packets += 1
    if first_pkt:
        first_pkt = False
        start_seq_no = seq_no
        end_seq_no = seq_no
        start_chan = freq_chan
        end_chan = freq_chan
    else:
        if freq_chan < start_chan:
            start_chan = freq_chan
        if seq_no < start_seq_no:
            start_seq_no = seq_no
        if freq_chan > end_chan:
            end_chan = freq_chan
        if seq_no > end_seq_no:
            end_seq_no = seq_no

print(f"Found {total_packets} packets")
print(
    f"Start time sample = {start_seq_no}, total time samples = {end_seq_no - start_seq_no + 1}, (= total time {1080e-9 * (end_seq_no - start_seq_no + 1)} seconds)"
)
print(
    f"Start channel = {start_chan}, total channels = {(end_chan - start_chan) + 1}"
)
total_channels = (end_chan - start_chan) + 1
total_time_packets = (end_seq_no - start_seq_no + 1) // 64
expected_packets = total_channels * total_time_packets
print(f"expected packets = {expected_packets}")

# Get all the data into a big numpy array
# ADCs x channels x time samples
all_samples = np.zeros(
    (total_ADCs, total_channels, (end_seq_no - start_seq_no + 64)),
    dtype=np.complex64,
)
all_samples_scaled = np.zeros(
    (total_ADCs, total_channels, (end_seq_no - start_seq_no + 64)),
    dtype=np.complex64,
)
# scale factor for each sample, initialise with -1
all_scales = -500 * np.ones(
    (total_ADCs, total_channels, (end_seq_no - start_seq_no + 64)), dtype=np.float32
)
pkt_scale = np.zeros(total_ADCs, dtype=np.float32)
pcount = 0
IS_FIRST_PACKET = True
for ts, pkt_payload in zip(tstamps, payloads):
    pcount += 1
    seq_no = struct.unpack("<Q", pkt_payload[0:8])[0]
    FPGA_id = struct.unpack("<I", pkt_payload[8:12])[0]
    freq_chan = struct.unpack("<H", pkt_payload[12:14])[0] - start_chan
    padding = struct.unpack("<Q", pkt_payload[14:22])[0]

    # Get scale factors
    for adc in range(total_ADCs):
        pkt_scale[adc] = np.float32(
            struct.unpack("H", pkt_payload[(22 + 2 * adc) : (24 + 2 * adc)])[0]
        )
    # Get data
    data_base = 22 + total_ADCs * 2
    for adc in range(total_ADCs):
        for t in range(64):  # 64 time samples per packet
            x_i, x_q = struct.unpack(
                ">bb",
                pkt_payload[
                    (data_base + t * total_ADCs * 2 + adc * 2) : (
                        data_base + t * total_ADCs * 2 + adc * 2 + 2
                    )
                ],
            )
            all_samples[adc, freq_chan, seq_no - start_seq_no + t] = (
             #   1j * np.float32(x_q) + np.float32(x_i)
                1j * x_q + x_i
            )
            # all_samples_scaled[adc, freq_chan, seq_no - start_seq_no + t] = (
            #     pkt_scale[adc] * (1j * np.float32(x_q) + np.float32(x_i))
            # )
            #all_scales[adc, freq_chan, seq_no - start_seq_no + t] = pkt_scale[adc]
    if pcount >= pcount_max:
        print(f"stopping packet decoding at packet {pcount}")
        break

all_samples = all_samples.transpose((1,0,2))

In [ ]:
import h5py

# Open file (read-only mode)
with h5py.File("first_buffer.hdf5", "r") as f:
    # List all groups
    print("Keys:", list(f.keys()))
    
    # Access a dataset
    dataset = f["packet_samples"]
    print("Shape:", dataset.shape)
    print("Data type:", dataset.dtype)
    
    # Load data into memory (NumPy array)
    data = dataset[:]

data = data.transpose((0,1,3,2))
data = data['r'].astype(np.float32) + 1j * data['i'].astype(np.float32)

In [ ]:
data[4][0][5]

In [ ]:
all_samples[4][5][:64]

In [ ]:
all_samples.shape

In [ ]:
data.shape

In [ ]:
for i in range(8):
    for j in range(20):
        for k in range(16):
            assert np.isclose(data[i][k][j], all_samples[i][j][k * 64: (k+1) * 64]).all()